In [2]:
import requests
from app.services.parsing_patientdata import PatientData
import numpy as np
import torch

ORTHANC_URL = "http://192.168.0.45:8042"
target_name = "뵹서준"

np.set_printoptions(threshold=np.inf)  # 전체 배열 출력
np.set_printoptions(linewidth=200)    # 한 줄 길이 설정

parsing_patient = PatientData(ORTHANC_URL)
patient_tag_data, studies = parsing_patient.parsing_target_patient(target_name)
studies_tag_data, series = parsing_patient.parsing_target_studies(studies)
instances_data, modality = parsing_patient.parsing_target_series(series)
instances_images = parsing_patient.parsing_target_dicom_image(instances_data)

print(studies)
print(patient_tag_data)
print(instances_data)
torch_image = torch.from_numpy(instances_images).permute(0,3,1,2)
print(torch_image.shape)
print(studies_tag_data['StudyDescription'])

['319710f8-9bfd9e36-a061bca7-479ce3da-eb700f9a']
{'PatientBirthDate': '2024-07-23', 'PatientID': '2663', 'PatientName': '뵹서준', 'PatientSex': 'M'}
['29089b35-98efe024-f53c6d52-a3df3087-8c1a7b10']
torch.Size([1, 3, 512, 512])
악성흑색종


In [3]:
import torchvision
import torch
model = torchvision.models.resnet18(pretrained = False)
model.fc = torch.nn.Linear(model.fc.in_features,15)
weight_path = 'app\models\Skin_RESNET_18.pth'
state_dict = torch.load(weight_path, map_location='cpu')
model.load_state_dict(state_dict)

c:\finalprjFastAPI\.venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\finalprjFastAPI\.venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

In [4]:
probs = torch.nn.functional.softmax(model(torch_image/255.0),dim=1)
print(probs*100)

tensor([[15.0297,  4.2034,  6.4438,  3.8213,  2.0636,  4.0239, 23.6326,  3.2112,
          6.3205,  3.6933,  2.0878,  7.3707,  3.1670,  9.4700,  5.4610]],
       grad_fn=<MulBackward0>)


In [7]:
class_map = {
    0: "광선각화증",
    1: "기저세포암",
    2: "멜라닌세포모반",
    3: "보웬병",
    4: "비립종",
    5: "사마귀",
    6: "악성흑생종",
    7: "지루각화증",
    8: "편평세포암",
    9: "표피낭종",
    10: "피부섬유종",
    11: "피지샘증식증",
    12: "혈관종",
    13: "화농육아종",
    14: "흑색점"
}
class_map[torch.argmax(probs).item()]

'악성흑생종'